Instalamos e importamos módulos de la api de procyclingstats


In [14]:
!pip install procyclingstats


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from procyclingstats import Race, RaceClimbs, Stage


In [16]:
# Importamos librerías
import os
import pandas as pd
from tqdm import tqdm

Vamos a sacar todos los puertos subidos en el Tour 2022(aun estamos probando funcionampiento de la API)

In [17]:
race_climbs = RaceClimbs("race/tour-de-france/2022/route/climbs")
race_climbs.climbs()

[{'climb_name': 'Col de la Croix de Fer',
  'climb_url': 'location/col-de-la-croix-de-fer',
  'length': 29.1,
  'steepness': 5.2,
  'top': 2068,
  'km_before_finnish': 55},
 {'climb_name': 'Col du Galibier',
  'climb_url': 'location/col-du-galibier',
  'length': 23.0,
  'steepness': 5.1,
  'top': 2619,
  'km_before_finnish': 132},
 {'climb_name': "Montée de l'altiport de Megève",
  'climb_url': 'location/montee-de-l-altiport-de-megeve',
  'length': 19.3,
  'steepness': 4.0,
  'top': 1370,
  'km_before_finnish': 2},
 {'climb_name': 'Col du Galibier',
  'climb_url': 'location/col-du-galibier',
  'length': 17.6,
  'steepness': 6.8,
  'top': 2630,
  'km_before_finnish': 45},
 {'climb_name': "Col d'Aubisque",
  'climb_url': 'location/col-d-aubisque',
  'length': 17.1,
  'steepness': 7.0,
  'top': 1703,
  'km_before_finnish': 66},
 {'climb_name': 'Pas de Morgins',
  'climb_url': 'location/pas-de-morgins',
  'length': 15.4,
  'steepness': 6.2,
  'top': 1376,
  'km_before_finnish': 10},
 {'cli

In [18]:
def cargar_datos_wikipedia():
    # Ruta de la carpeta donde están los archivos CSV
    ruta_datos = '../datos/'

    # Diccionario para almacenar los DataFrames
    diccionario_dataframes = {}

    # Iteramos sobre cada archivo en la carpeta
    for archivo in os.listdir(ruta_datos):
        if archivo.endswith('.csv'):
            clave_df = archivo.replace('.csv', '')
            ruta_archivo = os.path.join(ruta_datos, archivo)
            df = pd.read_csv(ruta_archivo, index_col=0)
            diccionario_dataframes[clave_df] = df
    return diccionario_dataframes

In [19]:
dicc_dataframes=cargar_datos_wikipedia()
[df_giro, df_tour, df_vuelta] = [dicc_dataframes["df_del_Giro_de_Italia"],
                                 dicc_dataframes["df_del_Tour_de_Francia"] ,
                                 dicc_dataframes["df_de_la_Vuelta_a_España"]]

In [20]:
df_tour.sample(10)

,Año,Ganador,Tiempo del ganador,Kilómetros totales,Velocidad media,N.º de etapas[1]​,N.º de corredores que participaron,N.º de corredores que finalizaron
Edición,,,,,,,,
102,2015,Chris Froome,84:46’14’’,3.360,"39,639 km/h",21.0,198,160
87,2000,Lance Armstrong[5]​,92:33’08’’,3.662,"39,569 km/h",21.0,177[6]​,127
21,1927,Nicolas Frantz,198:16’42’’,5.321,"27,224 km/h",24.0,142,39
97,2010,Andy Schleck[14]​,91:58’48’’,3.641,"39,584 km/h",21.0,197[15]​,170 (168)[14]​
41,1954,Louison Bobet,140:06’05’’,4.656,"33,229 km/h",25.0,110,69
75,1988,Pedro Delgado,84:27’58’’,3.286,"38,909 km/h",23.0,198,151
31,1937,Roger Lapébie,138:58’31’’,4.415,"31,768 km/h",31.0,98,46
110,2023,Jonas Vingegaard,82:05’42’’,"3,405.6","41,483 km/h",21.0,176,150
5,1907,Lucien Petit-Breton,47 puntos,4.488,"28,740 km/h",14.0,93,33


Eliminamos las filas en las que en las columna año aparece más de un año, pues son años en los que no se disputo la carrera, por Guerras u otros motivos

In [21]:
df_giro = df_giro[~df_giro['Año'].str.contains('-')]
df_tour = df_tour[~df_tour['Año'].str.contains('-')]
df_vuelta = df_vuelta[~df_vuelta['Año'].str.contains('-')]


In [22]:
lista_anios_giro = df_giro['Año'].tolist()
lista_anios_tour = df_tour['Año'].tolist()
lista_anios_vuelta = df_vuelta['Año'].tolist()



In [23]:
lista_ediciones_tres_carreras = [lista_anios_giro, lista_anios_tour, lista_anios_vuelta]

In [48]:
# Por cada una de las tres carreras (Giro, Tour, Vuelta) sacamos con la API de procyclingstats la informacion
# sobre los ganadores de cada etapa de cada edición
for lista_ediciones in tqdm(lista_ediciones_tres_carreras):
    ganadores_etapas_por_anio = []
    for anio in lista_anios_tour:
        race = Race(f"race/tour-de-france/{anio}")
        info_ganadores=race.stages_winners()
        ganadores_etapas_por_anio.append(info_ganadores)
       # ganadores_etapas_por_anio[anio] = info_ganadores
        



100%|██████████| 3/3 [01:10<00:00, 23.46s/it]


In [50]:
df_ganadores_de_etapa = pd.DataFrame(ganadores_etapas_por_anio)
df_ganadores_de_etapa.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,"{'rider_name': 'GARIN Maurice', 'rider_url': '...","{'rider_name': 'AUCOUTURIER Hippolyte', 'rider...","{'rider_name': 'AUCOUTURIER Hippolyte', 'rider...","{'rider_name': 'LAESER Charles', 'rider_url': ...","{'rider_name': 'GARIN Maurice', 'rider_url': '...","{'rider_name': 'GARIN Maurice', 'rider_url': '...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,"{'rider_name': 'GARIN Maurice', 'rider_url': '...","{'rider_name': 'AUCOUTURIER Hippolyte', 'rider...","{'rider_name': 'AUCOUTURIER Hippolyte', 'rider...","{'rider_name': 'POTHIER Lucien', 'rider_url': ...","{'rider_name': 'AUCOUTURIER Hippolyte', 'rider...","{'rider_name': 'AUCOUTURIER Hippolyte', 'rider...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,"{'rider_name': 'TROUSSELIER Louis', 'rider_url...","{'rider_name': 'AUCOUTURIER Hippolyte', 'rider...","{'rider_name': 'TROUSSELIER Louis', 'rider_url...","{'rider_name': 'AUCOUTURIER Hippolyte', 'rider...","{'rider_name': 'TROUSSELIER Louis', 'rider_url...","{'rider_name': 'DORTIGNACQ Jean-Baptiste', 'ri...","{'rider_name': 'TROUSSELIER Louis', 'rider_url...","{'rider_name': 'AUCOUTURIER Hippolyte', 'rider...","{'rider_name': 'TROUSSELIER Louis', 'rider_url...","{'rider_name': 'DORTIGNACQ Jean-Baptiste', 'ri...","{'rider_name': 'DORTIGNACQ Jean-Baptiste', 'ri...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,"{'rider_name': 'GEORGET Émile', 'rider_url': '...","{'rider_name': 'POTTIER René', 'rider_url': 'r...","{'rider_name': 'POTTIER René', 'rider_url': 'r...","{'rider_name': 'POTTIER René', 'rider_url': 'r...","{'rider_name': 'POTTIER René', 'rider_url': 'r...","{'rider_name': 'PASSERIEU Georges', 'rider_url...","{'rider_name': 'TROUSSELIER Louis', 'rider_url...","{'rider_name': 'DORTIGNACQ Jean-Baptiste', 'ri...","{'rider_name': 'TROUSSELIER Louis', 'rider_url...","{'rider_name': 'TROUSSELIER Louis', 'rider_url...","{'rider_name': 'TROUSSELIER Louis', 'rider_url...","{'rider_name': 'PASSERIEU Georges', 'rider_url...","{'rider_name': 'POTTIER René', 'rider_url': 'r...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,"{'rider_name': 'TROUSSELIER Louis', 'rider_url...","{'rider_name': 'GEORGET Émile', 'rider_url': '...","{'rider_name': 'GEORGET Émile', 'rider_url': '...","{'rider_name': 'CADOLLE Marcel', 'rider_url': ...","{'rider_name': 'GEORGET Émile', 'rider_url': '...","{'rider_name': 'PASSERIEU Georges', 'rider_url...","{'rider_name': 'GEORGET Émile', 'rider_url': '...","{'rider_name': 'GEORGET Émile', 'rider_url': '...","{'rider_name': 'PETIT-BRETON Lucien', 'rider_u...","{'rider_name': 'GARRIGOU Gustave', 'rider_url'...","{'rider_name': 'PETIT-BRETON Lucien', 'rider_u...","{'rider_name': 'GARRIGOU Gustave', 'rider_url'...","{'rider_name': 'GEORGET Émile', 'rider_url': '...","{'rider_name': 'PASSERIEU Georges', 'rider_url...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
# Inicializamos un diccionario para almacenar la información de cada año
for lista_ediciones in tqdm(lista_ediciones_tres_carreras):
    lista_informacion=[]
    for anio in lista_anios_tour:
        race = Race(f"race/tour-de-france/{anio}")
        # numero_etapas=len(race.stages())
        for i, etapa in enumerate(race.stages()):
            try:
                stage = Stage(etapa['stage_url'])
                lista_informacion.append(stage.parse())
                """
                ganadores_etapas_por_anio[anio] = info_ganadores
                print(f"En el año {anio} hubo los siguientes ganadores de_etapas")
                print(ganadores_etapas_por_anio[anio])       
                """
            except :
                print(f"Error al analizar la etapa en el año {anio}:")



  0%|          | 0/3 [00:00<?, ?it/s]

Error al analizar la etapa en el año 1971:
Error al analizar la etapa en el año 1978:
Error al analizar la etapa en el año 1978:
Error al analizar la etapa en el año 1982:
Error al analizar la etapa en el año 1995:
Error al analizar la etapa en el año 1998:


 33%|███▎      | 1/3 [12:38<25:16, 758.08s/it]

Error al analizar la etapa en el año 1971:
Error al analizar la etapa en el año 1978:
Error al analizar la etapa en el año 1978:
Error al analizar la etapa en el año 1982:
Error al analizar la etapa en el año 1995:
Error al analizar la etapa en el año 1998:


 67%|██████▋   | 2/3 [26:09<13:09, 789.64s/it]

Error al analizar la etapa en el año 1971:
Error al analizar la etapa en el año 1978:
Error al analizar la etapa en el año 1978:
Error al analizar la etapa en el año 1982:
Error al analizar la etapa en el año 1995:
Error al analizar la etapa en el año 1998:


100%|██████████| 3/3 [39:28<00:00, 789.64s/it]


In [29]:
pd.set_option('display.max_columns', None)

In [ ]:
#Creamos un DataFrame con la informacion que nos ha devuelto la API sobe las etapas
df_etapas=pd.DataFrame(lista_informacion)
df_etapas.head()

,arrival,avg_speed_winner,avg_temperature,climbs,date,departure,distance,gc,is_one_day_race,kom,pcs_points_scale,points,profile_icon,profile_score,race_category,race_startlist_quality_score,results,stage_type,start_time,teams,uci_points_scale,vertical_meters,won_how,youth
0,Lyon,26.300,NaN,[],1903-07-01,Montgeron,467.0,"[{'rider_name': 'GARIN Maurice', 'rider_url': ...",False,[],GT.A.Stage,[],p1,NaN,ME - Men Elite,310,"[{'rider_name': 'GARIN Maurice', 'rider_url': ...",RR,-,[],,NaN,0.2 km solo,[]
1,Marseille,25.830,NaN,[],1903-07-05,Lyon,374.0,"[{'rider_name': 'GARIN Maurice', 'rider_url': ...",False,[],GT.A.Stage,[],p4,NaN,ME - Men Elite,310,"[{'rider_name': 'AUCOUTURIER Hippolyte', 'ride...",RR,-,[],,NaN,Sprint à deux,[]
2,Toulouse,23.610,NaN,[],1903-07-08,Marseille,423.0,"[{'rider_name': 'GARIN Maurice', 'rider_url': ...",False,[],GT.A.Stage,[],p1,NaN,ME - Men Elite,310,"[{'rider_name': 'AUCOUTURIER Hippolyte', 'ride...",RR,-,[],,NaN,? km solo,[]
3,Bordeaux,30.570,NaN,[],1903-07-12,Toulouse,268.0,"[{'rider_name': 'GARIN Maurice', 'rider_url': ...",False,[],GT.A.Stage,[],p1,NaN,ME - Men Elite,310,"[{'rider_name': 'LAESER Charles', 'rider_url':...",RR,-,[],,NaN,? km solo,[]
4,Nantes,25.849,NaN,[],1903-07-13,Bordeaux,425.0,"[{'rider_name': 'GARIN Maurice', 'rider_url': ...",False,[],GT.A.Stage,[],p1,NaN,ME - Men Elite,310,"[{'rider_name': 'GARIN Maurice', 'rider_url': ...",RR,-,[],,NaN,? - let us know!,[]


In [ ]:
# Eliminamos una serie de columnas que no son de interés para nuestro análisis
df_etapas.drop(["avg_temperature","gc","is_one_day_race","kom","pcs_points_scale", "points", "profile_icon", "profile_score", "race_category",	"race_startlist_quality_score", "start_time", "teams", "uci_points_scale", "won_how", "youth"], axis=1, inplace= True)

In [46]:
df_etapas.head()

,arrival,avg_speed_winner,climbs,date,departure,distance,results,stage_type,vertical_meters
0,Lyon,26.300,[],1903-07-01,Montgeron,467.0,"[{'rider_name': 'GARIN Maurice', 'rider_url': ...",RR,NaN
1,Marseille,25.830,[],1903-07-05,Lyon,374.0,"[{'rider_name': 'AUCOUTURIER Hippolyte', 'ride...",RR,NaN
2,Toulouse,23.610,[],1903-07-08,Marseille,423.0,"[{'rider_name': 'AUCOUTURIER Hippolyte', 'ride...",RR,NaN
3,Bordeaux,30.570,[],1903-07-12,Toulouse,268.0,"[{'rider_name': 'LAESER Charles', 'rider_url':...",RR,NaN
4,Nantes,25.849,[],1903-07-13,Bordeaux,425.0,"[{'rider_name': 'GARIN Maurice', 'rider_url': ...",RR,NaN


In [ ]:
#Vamos a limpiar la columna "results"
df_etapas["results"]

0       [{'rider_name': 'GARIN Maurice', 'rider_url': ...
1       [{'rider_name': 'AUCOUTURIER Hippolyte', 'ride...
2       [{'rider_name': 'AUCOUTURIER Hippolyte', 'ride...
3       [{'rider_name': 'LAESER Charles', 'rider_url':...
4       [{'rider_name': 'GARIN Maurice', 'rider_url': ...
                              ...                        
2372    [{'rider_name': 'CARAPAZ Richard', 'rider_url'...
2373    [{'rider_name': 'CAMPENAERTS Victor', 'rider_u...
2374    [{'rider_name': 'POGAČAR Tadej', 'rider_url': ...
2375    [{'rider_name': 'POGAČAR Tadej', 'rider_url': ...
2376    [{'rider_name': 'POGAČAR Tadej', 'rider_url': ...
Name: results, Length: 2377, dtype: object

In [53]:
df_etapas['results'] = df_etapas['results'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)
df_etapas.head()

,arrival,avg_speed_winner,climbs,date,departure,distance,results,stage_type,vertical_meters
0,Lyon,26.300,[],1903-07-01,Montgeron,467.0,"{'rider_name': 'GARIN Maurice', 'rider_url': '...",RR,NaN
1,Marseille,25.830,[],1903-07-05,Lyon,374.0,"{'rider_name': 'AUCOUTURIER Hippolyte', 'rider...",RR,NaN
2,Toulouse,23.610,[],1903-07-08,Marseille,423.0,"{'rider_name': 'AUCOUTURIER Hippolyte', 'rider...",RR,NaN
3,Bordeaux,30.570,[],1903-07-12,Toulouse,268.0,"{'rider_name': 'LAESER Charles', 'rider_url': ...",RR,NaN
4,Nantes,25.849,[],1903-07-13,Bordeaux,425.0,"{'rider_name': 'GARIN Maurice', 'rider_url': '...",RR,NaN


In [42]:
df_etapas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2377 entries, 0 to 2376
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   arrival           2377 non-null   object 
 1   avg_speed_winner  2377 non-null   float64
 2   date              2377 non-null   object 
 3   departure         2377 non-null   object 
 4   distance          2377 non-null   float64
 5   results           2377 non-null   object 
 6   stage_type        2377 non-null   object 
 7   vertical_meters   1625 non-null   float64
dtypes: float64(3), object(5)
memory usage: 148.7+ KB


In [43]:
nan_count = df_etapas['vertical_meters'].isna().sum()
nan_count

np.int64(752)